In [5]:
import pathlib
import pickle


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option('display.max_rows', 500)

DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

processed_file_path = DATA_DIR / 'processed' / 'ames_with_correct_types.pkl'

with open(processed_file_path, 'rb') as file:
    (
        data,
        continuous_variables,
        discrete_variables,
        ordinal_variables,
        categorical_variables,
    ) = pickle.load(file)

def plot_categoricals(data, cols, sorted=True):
    summary = data[cols] \
        .describe() \
        .transpose() \
        .sort_values(by='count')

    print(summary)

    for k, (col, val) in enumerate(summary['count'].items()):
        plt.figure()
        ser = data[col].value_counts()
        if sorted:
            ser = ser.sort_values()
        else:
            ser = ser.sort_index()
        ax = ser.plot.barh()
        for container in ax.containers:
            ax.bar_label(container)
        plt.title(f'{col}, n={int(val)}')
        plt.show()

data['MS.Zoning'].unique()
data['MS.Zoning'].value_counts()
selection = ~(data['MS.Zoning'].isin(['A (agr)', 'C (all)', 'I (all)']))
selection.value_counts()
data = data[selection]
data['MS.Zoning'] = data['MS.Zoning'].cat.remove_unused_categories()
data['MS.Zoning'].value_counts()

data['Sale.Type'].value_counts()
data['Sale.Type'].unique()

processed_data = data.copy()

def remap_categories(
    series: pd.Series,
    old_categories: tuple[str],
    new_category: str,
) -> pd.Series:
    # Add the new category to the list of valid categories.
    series = series.cat.add_categories(new_category)

    # Set all items of the old categories as the new category.
    remapped_items = series.isin(old_categories)
    series.loc[remapped_items] = new_category

    # Clean up the list of categories, the old categories no longer exist.
    series = series.cat.remove_unused_categories()

    return series

processed_data['Sale.Type'] = remap_categories(
    series=processed_data['Sale.Type'],
    old_categories=('WD ', 'CWD', 'VWD'),
    new_category='GroupedWD',
)

processed_data['Sale.Type'] = remap_categories(
    series=processed_data['Sale.Type'],
    old_categories=('COD', 'ConLI', 'Con', 'ConLD', 'Oth', 'ConLw'),
    new_category='Other',
)

processed_data['Sale.Type'].value_counts()

data = processed_data
data['Street'].value_counts()
data = data.drop(columns='Street')
data['Condition.1'].value_counts()
data['Condition.2'].value_counts()
pd.crosstab(data['Condition.1'], data['Condition.2'])

processed_data = data.copy()
for col in ('Condition.1', 'Condition.2'):
    processed_data[col] = remap_categories(
        series=processed_data[col],
        old_categories=('RRAn', 'RRAe', 'RRNn', 'RRNe'),
        new_category='Railroad',
    )
    processed_data[col] = remap_categories(
        series=processed_data[col],
        old_categories=('Feedr', 'Artery'),
        new_category='Roads',
    )
    processed_data[col] = remap_categories(
        series=processed_data[col],
        old_categories=('PosA', 'PosN'),
        new_category='Positive',
    )
processed_data['Condition.1'].value_counts()
processed_data['Condition.2'].value_counts()
pd.crosstab(processed_data['Condition.1'], processed_data['Condition.2'])
processed_data['Condition'] = pd.Series(
    index=processed_data.index,
    dtype=pd.CategoricalDtype(categories=(
        'Norm',
        'Railroad',
        'Roads',
        'Positive',
        'RoadsAndRailroad',
    )),
)

norm_items = processed_data['Condition.1'] == 'Norm'
processed_data['Condition'][norm_items] = 'Norm'

railroad_items = \
    (processed_data['Condition.1'] == 'Railroad') \
    & (processed_data['Condition.2'] == 'Norm')
processed_data['Condition'][railroad_items] = 'Railroad'

roads_items = \
    (processed_data['Condition.1'] == 'Roads') \
    & (processed_data['Condition.2'] != 'Railroad')
processed_data['Condition'][roads_items] = 'Roads'

positive_items = processed_data['Condition.1'] == 'Positive'
processed_data['Condition'][positive_items] = 'Positive'

roads_and_railroad_items = \
    ( \
        (processed_data['Condition.1'] == 'Railroad') \
        & (processed_data['Condition.2'] == 'Roads')
    ) \
    | ( \
        (processed_data['Condition.1'] == 'Roads') \
        & (processed_data['Condition.2'] == 'Railroad') \
    )
processed_data['Condition'][roads_and_railroad_items] = 'RoadsAndRailroad'

processed_data['Condition'].value_counts()

processed_data = processed_data.drop(columns=['Condition.1', 'Condition.2'])

data = processed_data

data['HasShed'] = data['Misc.Feature'] == 'Shed'
data = data.drop(columns='Misc.Feature')

data['HasShed'].value_counts()

data['HasAlley'] = ~data['Alley'].isna()
data = data.drop(columns='Alley')

data['HasAlley'].value_counts()

plot_categoricals(data, ['Exterior.1st', 'Exterior.2nd'])

data['Exterior.2nd'] = remap_categories(
    series=data['Exterior.2nd'],
    old_categories=('Brk Cmn', ),
    new_category='BrkComm',
)
data['Exterior.2nd'] = remap_categories(
    series=data['Exterior.2nd'],
    old_categories=('CmentBd', ),
    new_category='CemntBd',
)
data['Exterior.2nd'] = remap_categories(
    series=data['Exterior.2nd'],
    old_categories=('Wd Shng', ),
    new_category='WdShing',
)

for col in ('Exterior.1st', 'Exterior.2nd'):
    categories = data[col].cat.categories
    data[col] = data[col].cat.reorder_categories(sorted(categories))


pd.crosstab(data['Exterior.1st'], data['Exterior.2nd'])

processed_data = data.copy()

mat_count = processed_data['Exterior.1st'].value_counts()
mat_count

rare_materials = list(mat_count[mat_count < 40].index)
rare_materials

processed_data['Exterior'] = remap_categories(
    series=processed_data['Exterior.1st'],
    old_categories=rare_materials,
    new_category='Other',
)
processed_data = processed_data.drop(columns=['Exterior.1st', 'Exterior.2nd'])

processed_data['Exterior'].value_counts()

data = processed_data
data = data.drop(columns='Heating')

plot_categoricals(data, ['Roof.Matl', 'Roof.Style'])
data = data.drop(columns='Roof.Matl')

data['Roof.Style'] = remap_categories(
    series=data['Roof.Style'],
    old_categories=[
        'Flat',
        'Gambrel',
        'Mansard',
        'Shed',
    ],
    new_category='Other',
)

data['Roof.Style'].value_counts()

data['Mas.Vnr.Type'].info()

data['Mas.Vnr.Type'].value_counts()

data['Mas.Vnr.Type'] = remap_categories(
    series=data['Mas.Vnr.Type'],
    old_categories=[
        'BrkCmn',
        'CBlock',
    ],
    new_category='Other',
)

data['Mas.Vnr.Type'] = data['Mas.Vnr.Type'].cat.add_categories('None')
data['Mas.Vnr.Type'][data['Mas.Vnr.Type'].isna()] = 'None'

data['Mas.Vnr.Type'].value_counts()

plot_categoricals(data, ['MS.SubClass'])

data['MS.SubClass'] = remap_categories(
    series=data['MS.SubClass'],
    old_categories=[75, 45, 180, 40, 150],
    new_category='Other',
)

data['MS.SubClass'].value_counts()

data['Foundation'] = remap_categories(
    series=data['Foundation'],
    old_categories=['Slab', 'Stone', 'Wood'],
    new_category='Other',
)

data['Neighborhood'].value_counts()

selection = ~data['Neighborhood'].isin([
    'Blueste',
    'Greens',
    'GrnHill',
    'Landmrk',
])
data = data[selection]

data['Neighborhood'] = data['Neighborhood'].cat.remove_unused_categories()

data['Neighborhood'].value_counts()

data['Garage.Type'].info()
data['Garage.Type'].value_counts()

data['Garage.Type'] = data['Garage.Type'].cat.add_categories(['NoGarage'])
data['Garage.Type'][data['Garage.Type'].isna()] = 'NoGarage'

data['Garage.Type'].value_counts()

all_categorical = data.select_dtypes('category').columns

new_categorical_variables = [ \
    col for col in all_categorical \
    if not col in ordinal_variables \
]

data = data.drop(columns='Utilities')

data = data.drop(columns='Pool.QC')

data['Fence'].value_counts().sort_index()

old_categories = list(data['Fence'].cat.categories)
old_categories

new_categories = old_categories + ['NoFence']
new_categories

data['Fence'] = data['Fence'].cat.set_categories(new_categories)

data['Fence'].dtype

data['Fence'][data['Fence'].isna()] = 'NoFence'

data['Fence'].value_counts().sort_index()

data['Fireplace.Qu'].value_counts().sort_index()

data['Fireplaces'].value_counts()

data = data.drop(columns='Fireplace.Qu')

data = data.drop(columns=['Garage.Cond', 'Garage.Qual'])

data['Garage.Finish'] = data['Garage.Finish'] \
    .cat \
    .as_unordered() \
    .cat \
    .add_categories(['NoGarage'])
data['Garage.Finish'][data['Garage.Finish'].isna()] = 'NoGarage'

data['Garage.Finish'].value_counts()

data['Garage.Finish'].dtype

data['Garage.Finish'].cat.ordered

data['Electrical'].isna().value_counts()

plot_categoricals(data, ['Electrical'], sorted=False)

data['Electrical'][data['Electrical'].isna()] = 'SBrkr'

ordinal_columns = [col for col in data.select_dtypes('category') if data[col].cat.ordered]

data[ordinal_columns].info()

data['Bsmt.Exposure'].unique()

data['Bsmt.Exposure'][data['Bsmt.Exposure'].isna()] = 'NA'
data['Bsmt.Exposure'] = data['Bsmt.Exposure'] \
    .cat \
    .as_unordered() \
    .cat \
    .remove_unused_categories()

for col in ('Bsmt.Qual', 'Bsmt.Cond', 'BsmtFin.Type.1', 'BsmtFin.Type.2'):
    data[col] = data[col].cat.add_categories(['NA'])
    data[col][data[col].isna()] = 'NA'
    data[col] = data[col] \
        .cat \
        .as_unordered() \
        .cat \
        .remove_unused_categories()
    
data['Bsmt.Cond'][data['Bsmt.Cond'] == 'Po'] = 'Fa'
data['Bsmt.Cond'][data['Bsmt.Cond'] == 'Ex'] = 'Gd'
data['Bsmt.Cond'] = data['Bsmt.Cond'].cat.remove_unused_categories()

data['Bsmt.Cond'].value_counts()

data[ordinal_columns].info()

def plot_numericals(data, cols):
    summary = data[cols] \
        .describe() \
        .transpose() \
        .sort_values(by='count')

    print(summary)

    n = data.shape[0]
    b = int(np.sqrt(n))
    for k, (col, val) in enumerate(summary['count'].items()):
        plt.figure()
        data[col].plot.hist(bins=b)
        plt.title(f'{col}, n={int(val)}')
        plt.show()


data['SalePrice'].describe()

data['SalePrice'] = data['SalePrice'].apply(np.log10)

data['SalePrice'].describe()

data['Lot.Frontage'].info()

missing_lot_frontage = data['Lot.Frontage'].isna()

data['MS.SubClass'][missing_lot_frontage].value_counts()

data['Lot.Config'][missing_lot_frontage].value_counts()

data['Land.Contour'][missing_lot_frontage].value_counts()

data[['Lot.Frontage', 'Lot.Area']].corr()
aux_data = data[['Lot.Frontage', 'Lot.Area']].copy()
aux_data['Sqrt.Lot.Area'] = aux_data['Lot.Area'].apply(np.sqrt)

aux_data = data[['Lot.Frontage', 'Lot.Area']].copy()
aux_data['Sqrt.Lot.Area'] = aux_data['Lot.Area'].apply(np.sqrt)

aux_data[['Lot.Frontage', 'Sqrt.Lot.Area']].corr()

data['Lot.Frontage'] = data['Lot.Frontage'].fillna(data['Lot.Frontage'].median())

data['Lot.Frontage'].info()

data['Garage.Yr.Blt'].describe()

garage_age = data['Yr.Sold'] - data['Garage.Yr.Blt']
garage_age.describe()

data[garage_age < 0.0].transpose()

garage_age[garage_age < 0.0] = 0.0

data = data.drop(columns='Garage.Yr.Blt')
data['Garage.Age'] = garage_age

data['Garage.Age'].info()

data['Garage.Type'][data['Garage.Age'].isna()].value_counts()

data['Garage.Age'] = data['Garage.Age'].fillna(data['Garage.Age'].median())

data[['Year.Remod.Add', 'Year.Built', 'Yr.Sold']].describe()

remod_age = data['Yr.Sold'] - data['Year.Remod.Add']
remod_age.describe()

data[remod_age < 0.0].transpose()

remod_age[remod_age < 0.0] = 0.0

house_age = data['Yr.Sold'] - data['Year.Built']
house_age.describe()

data[house_age < 0.0].transpose()

house_age[house_age < 0.0] = 0.0

data = data.drop(columns=['Year.Remod.Add', 'Year.Built'])
data['Remod.Age'] = remod_age
data['House.Age'] = house_age

data['Mas.Vnr.Area'].info()

data['Mas.Vnr.Type'][data['Mas.Vnr.Area'].isna()].value_counts()

data.loc[data['Mas.Vnr.Area'].isna(), 'Mas.Vnr.Area'] = 0.0

num_houses = data.shape[0]
num_houses_with_pool = data[data['Pool.Area'] > 0].shape[0]
print(f'Out of {num_houses} houses, only {num_houses_with_pool} have a pool.')

data.info()
data = data.dropna(axis=0)
data.info()

for col in data.select_dtypes('category').columns:
    data[col] = data[col].cat.remove_unused_categories()

numerical_data = data.select_dtypes('number').drop(columns='SalePrice').copy()
target = data['SalePrice'].copy()

numerical_data.corrwith(target).sort_values()

categorical_columns = data.select_dtypes('category').columns

corr = data.corr(numeric_only=True)
corr

clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'

with open(clean_data_path, 'wb') as file:
    pickle.dump(data, file)

In [6]:
data.info

<bound method DataFrame.info of      MS.SubClass MS.Zoning  Lot.Frontage  Lot.Area Street Alley Lot.Shape  \
0             20        RL         141.0   31770.0   Pave   NaN       IR1   
1             20        RH          80.0   11622.0   Pave   NaN       Reg   
2             20        RL          81.0   14267.0   Pave   NaN       IR1   
3             20        RL          93.0   11160.0   Pave   NaN       Reg   
4             60        RL          74.0   13830.0   Pave   NaN       IR1   
...          ...       ...           ...       ...    ...   ...       ...   
2925          80        RL          37.0    7937.0   Pave   NaN       IR1   
2926          20        RL           NaN    8885.0   Pave   NaN       IR1   
2927          85        RL          62.0   10441.0   Pave   NaN       Reg   
2928          20        RL          77.0   10010.0   Pave   NaN       Reg   
2929          60        RL          74.0    9627.0   Pave   NaN       Reg   

     Land.Contour Utilities Lot.Config  ...